In [1]:
import pandas as pd
import os 
import matplotlib.pyplot as plt 
import re
import numpy as np

In [2]:
# load tree data from the municipality of Copenhagen 
cols = ['id','ny_dm_element','slaegtsnavn','planteaar','bydelsnavn','vejnavn','placering','wkb_geometry']
df = pd.read_csv('raw_data/trae_basis.csv',sep=',',usecols=cols)

# prepare tree data for analysis
df = df[df["planteaar"] >= 2000] # get data from 2000 and after 
df = df.dropna(subset=['planteaar']) # remove samples with unknown plant year 
df.columns = ['id','type','species','year','district','street','street_location','coordinates']


In [3]:
# Get the X and Y coordinates for geodata plot
def get_x(row):
    x = re.findall(r"[-+]?\d*\.\d+|\d+", row)[0]
    return float(x)
def get_y(row):
    y = re.findall(r"[-+]?\d*\.\d+|\d+", row)[1]
    return float(y)


df['X'] = df['coordinates'].apply(get_x)
df['Y'] = df['coordinates'].apply(get_y)
df = df.drop(columns = ["coordinates"])


# save tree data for exploratory tree 
df.to_csv('data/df_tree.csv', index=False) 